<a href="https://colab.research.google.com/github/Mc4minta/AIB5-PcapAttackClassifier/blob/main/DataProcessingRandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount google drive to access files in it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


import labelled csv from CICIDS2017 dataset

In [12]:
import pandas as pd
import numpy as np
import os

csv_dir = '/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/GeneratedLabelledFlows/TrafficLabelling'

csv_files = [
    'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',     # Benign, DDOS,
    'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', # Benign, Portscan
    'Monday-WorkingHours.pcap_ISCX.csv',                    # Benign
    'Tuesday-WorkingHours.pcap_ISCX.csv',                   # Benign, FTP, SSH
    'Wednesday-workingHours.pcap_ISCX.csv',                 # Benign, DOSs, Heartbleed

    # Fix : Replace "–" (Dashes) with "-" (Hyphens) to prevent decoding error
    'Thursday-WorkingHours-Morning-WebAttacks-Fix.pcap_ISCX.csv' # Benign, Web attacks
]

dfs = []

for file in csv_files:
    file_path = os.path.join(csv_dir, file)
    try:
        df = pd.read_csv(file_path)

        # Clean column names by replacing spaces with "_" (underscore)
        df.columns = df.columns.str.strip().str.replace(' ', '_')

        # Drop 65% of Benign class data randomly to fix class imbalance
        benign_df = df[df['Label'] == 'BENIGN']
        non_benign_df = df[df['Label'] != 'BENIGN']
        dropped_benign = benign_df.sample(frac=0.35, random_state=42)
        reduced_df = pd.concat([dropped_benign, non_benign_df], ignore_index=True)
        dfs.append(reduced_df)
        print(f"Loaded and reduced BENIGN in: {file}")

    except FileNotFoundError:
        print(f"File not found: {file}")
    except Exception as e:
        print(f"Error with {file}: {e}")

# Combine all dataframes as one
df = pd.concat(dfs, ignore_index=True)

# Drop unneeded columns
columns_to_drop = [
    'Flow_ID', 'Source_IP', 'Destination_IP', 'Protocol', 'Timestamp', 'Fwd_Header_Length.1',
]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Remove infinite or missing value rows
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna()

df.head()

Loaded and reduced BENIGN in: Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Loaded and reduced BENIGN in: Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
Loaded and reduced BENIGN in: Monday-WorkingHours.pcap_ISCX.csv
Loaded and reduced BENIGN in: Tuesday-WorkingHours.pcap_ISCX.csv
Loaded and reduced BENIGN in: Wednesday-workingHours.pcap_ISCX.csv
Loaded and reduced BENIGN in: Thursday-WorkingHours-Morning-WebAttacks-Fix.pcap_ISCX.csv


,Source_Port,Destination_Port,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,...,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label
0,80,40474,7182757,1,5,6.0,30.0,6.0,6.0,6.0,...,20,45201.000,0.000,45201.0,45201.0,7137556.0,0.00,7137556.0,7137556.0,BENIGN
1,80,28907,938509,1,5,6.0,30.0,6.0,6.0,6.0,...,20,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,BENIGN
2,0,0,119995732,123,0,0.0,0.0,0.0,0.0,0.0,...,0,3753432.167,5183854.063,13200000.0,1.0,13100000.0,6507819.26,26300000.0,6484077.0,BENIGN
3,58168,443,142954,10,6,553.0,3949.0,196.0,0.0,55.3,...,20,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,BENIGN
4,80,17541,2548725,1,6,6.0,36.0,6.0,6.0,6.0,...,20,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,BENIGN


In [13]:
# Rename all columns to more readable format
columns_mapping = {
    'Src_Port': 'Source_Port',
    'Dst_Port': 'Destination_Port',
    'Flow_Duration': 'Flow_Duration',
    'Tot_Fwd_Pkts': 'Total_Fwd_Packets',
    'Tot_Bwd_Pkts': 'Total_Backward_Packets',
    'TotLen_Fwd_Pkts': 'Total_Length_of_Fwd_Packets',
    'TotLen_Bwd_Pkts': 'Total_Length_of_Bwd_Packets',
    'Fwd_Pkt_Len_Max': 'Fwd_Packet_Length_Max',
    'Fwd_Pkt_Len_Min': 'Fwd_Packet_Length_Min',
    'Fwd_Pkt_Len_Mean': 'Fwd_Packet_Length_Mean',
    'Fwd_Pkt_Len_Std': 'Fwd_Packet_Length_Std',
    'Bwd_Pkt_Len_Max': 'Bwd_Packet_Length_Max',
    'Bwd_Pkt_Len_Min': 'Bwd_Packet_Length_Min',
    'Bwd_Pkt_Len_Mean': 'Bwd_Packet_Length_Mean',
    'Bwd_Pkt_Len_Std': 'Bwd_Packet_Length_Std',
    'Flow_Byts/s': 'Flow_Bytes/s',
    'Flow_Pkts/s': 'Flow_Packets/s',
    'Flow_IAT_Mean': 'Flow_IAT_Mean',
    'Flow_IAT_Std': 'Flow_IAT_Std',
    'Flow_IAT_Max': 'Flow_IAT_Max',
    'Flow_IAT_Min': 'Flow_IAT_Min',
    'Fwd_IAT_Tot': 'Fwd_IAT_Total',
    'Fwd_IAT_Mean': 'Fwd_IAT_Mean',
    'Fwd_IAT_Std': 'Fwd_IAT_Std',
    'Fwd_IAT_Max': 'Fwd_IAT_Max',
    'Fwd_IAT_Min': 'Fwd_IAT_Min',
    'Bwd_IAT_Tot': 'Bwd_IAT_Total',
    'Bwd_IAT_Mean': 'Bwd_IAT_Mean',
    'Bwd_IAT_Std': 'Bwd_IAT_Std',
    'Bwd_IAT_Max': 'Bwd_IAT_Max',
    'Bwd_IAT_Min': 'Bwd_IAT_Min',
    'Fwd_PSH_Flags': 'Fwd_PSH_Flags',
    'Bwd_PSH_Flags': 'Bwd_PSH_Flags',
    'Fwd_URG_Flags': 'Fwd_URG_Flags',
    'Bwd_URG_Flags': 'Bwd_URG_Flags',
    'Fwd_Header_Len': 'Fwd_Header_Length',
    'Bwd_Header_Len': 'Bwd_Header_Length',
    'Fwd_Pkts/s': 'Fwd_Packets/s',
    'Bwd_Pkts/s': 'Bwd_Packets/s',
    'Pkt_Len_Min': 'Min_Packet_Length',
    'Pkt_Len_Max': 'Max_Packet_Length',
    'Pkt_Len_Mean': 'Packet_Length_Mean',
    'Pkt_Len_Std': 'Packet_Length_Std',
    'Pkt_Len_Var': 'Packet_Length_Variance',
    'FIN_Flag_Cnt': 'FIN_Flag_Count',
    'SYN_Flag_Cnt': 'SYN_Flag_Count',
    'RST_Flag_Cnt': 'RST_Flag_Count',
    'PSH_Flag_Cnt': 'PSH_Flag_Count',
    'ACK_Flag_Cnt': 'ACK_Flag_Count',
    'URG_Flag_Cnt': 'URG_Flag_Count',
    'CWE_Flag_Count': 'CWE_Flag_Count',
    'ECE_Flag_Cnt': 'ECE_Flag_Count',
    'Down/Up_Ratio': 'Down/Up_Ratio',
    'Pkt_Size_Avg': 'Average_Packet_Size',
    'Fwd_Seg_Size_Avg': 'Avg_Fwd_Segment_Size',
    'Bwd_Seg_Size_Avg': 'Avg_Bwd_Segment_Size',
    'Fwd_Byts/b_Avg': 'Fwd_Byts/b_Avg',
    'Fwd_Pkts/b_Avg': 'Fwd_Pkts/b_Avg',
    'Fwd_Blk_Rate_Avg': 'Fwd_Blk_Rate_Avg',
    'Bwd_Byts/b_Avg': 'Bwd_Byts/b_Avg',
    'Bwd_Pkts/b_Avg': 'Bwd_Pkts/b_Avg',
    'Bwd_Blk_Rate_Avg': 'Bwd_Blk_Rate_Avg',
    'Subflow_Fwd_Pkts': 'Subflow_Fwd_Pkts',
    'Subflow_Fwd_Byts': 'Subflow_Fwd_Byts',
    'Subflow_Bwd_Pkts': 'Subflow_Bwd_Pkts',
    'Subflow_Bwd_Byts': 'Subflow_Bwd_Byts',
    'Init_Fwd_Win_Byts': 'Init_Fwd_Win_Byts',
    'Init_Bwd_Win_Byts': 'Init_Bwd_Win_Byts',
    'Fwd_Act_Data_Pkts': 'act_data_pkt_fwd',
    'Fwd_Seg_Size_Min': 'Fwd_Seg_Size_Min',
    'Active_Mean': 'Active_Mean',
    'Active_Std': 'Active_Std',
    'Active_Max': 'Active_Max',
    'Active_Min': 'Active_Min',
    'Idle_Mean': 'Idle_Mean',
    'Idle_Std': 'Idle_Std',
    'Idle_Max': 'Idle_Max',
    'Idle_Min': 'Idle_Min'
}


df = df.rename(columns=columns_mapping)
df = df.dropna()
df.head()

,Source_Port,Destination_Port,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,...,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label
0,80,40474,7182757,1,5,6.0,30.0,6.0,6.0,6.0,...,20,45201.000,0.000,45201.0,45201.0,7137556.0,0.00,7137556.0,7137556.0,BENIGN
1,80,28907,938509,1,5,6.0,30.0,6.0,6.0,6.0,...,20,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,BENIGN
2,0,0,119995732,123,0,0.0,0.0,0.0,0.0,0.0,...,0,3753432.167,5183854.063,13200000.0,1.0,13100000.0,6507819.26,26300000.0,6484077.0,BENIGN
3,58168,443,142954,10,6,553.0,3949.0,196.0,0.0,55.3,...,20,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,BENIGN
4,80,17541,2548725,1,6,6.0,36.0,6.0,6.0,6.0,...,20,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,BENIGN


In [14]:
# Remove unused attack classes
values_to_remove = ['Heartbleed']
df = df[~df['Label'].isin(values_to_remove)]
df = df.reset_index(drop=True)
df['Label'].value_counts()

,count
Label,
BENIGN,627922
DoS Hulk,230124
PortScan,158804
DDoS,128025
DoS GoldenEye,10293
FTP-Patator,7935
SSH-Patator,5897
DoS slowloris,5796
DoS Slowhttptest,5499


In [15]:
# Groupped DOS Labels and rename FTP and SSH to be bruteforce
label_mapping = {
    'BENIGN': 'Benign',
    'DoS Hulk': 'DoS',
    'DDoS': 'DDoS',
    'DoS GoldenEye': 'DoS',
    'DoS slowloris': 'DoS',
    'DoS Slowhttptest': 'DoS',
    'PortScan' : 'PortScan',
    'FTP-Patator': 'FTP-Bruteforce',
    'SSH-Patator': 'SSH-Bruteforce',
    'Web Attack - Brute Force' : 'Web-Bruteforce',
    'Web Attack - XSS' : 'XSS',
    'Web Attack - Sql Injection' : 'SQLi',
}

# Apply the label mapping to the 'Label' column
df['Label'] = df['Label'].map(label_mapping)
df['Label'].value_counts()

,count
Label,
Benign,627922
DoS,251712
PortScan,158804
DDoS,128025
FTP-Bruteforce,7935
SSH-Bruteforce,5897
Web-Bruteforce,1507
XSS,652
SQLi,21


In [16]:
# Remove duplicated benign columns
df = df[~((df['Label'] == 'Benign') & df.duplicated())]
df['Label'].value_counts()

,count
Label,
Benign,626333
DoS,251712
PortScan,158804
DDoS,128025
FTP-Bruteforce,7935
SSH-Bruteforce,5897
Web-Bruteforce,1507
XSS,652
SQLi,21


In [17]:
# Ccompare the count of benign label row and all other label row combine

benign_count = len(df[df['Label'] == 'Benign'])
other_labels_count = len(df[df['Label'] != 'Benign'])

print(f"Benign: {benign_count}")
print(f"Attack: {other_labels_count}")

Benign: 626333
Attack: 554553


In [18]:
# prompt: print value count of the destination port of the row of each labels of df all of them

for label in df['Label'].unique():
    print(f"Value counts for Destination_Port for label '{label}':")
    print(df[df['Label'] == label]['Destination_Port'].value_counts())
    print("-" * 40) # Separator between labels


Value counts for Destination_Port for label 'Benign':
Destination_Port
53       269662
443      144475
80        65987
123        6532
22         2970
          ...  
32933         1
45651         1
3768          1
1209          1
3321          1
Name: count, Length: 37968, dtype: int64
----------------------------------------
Value counts for Destination_Port for label 'DDoS':
Destination_Port
80       128022
64869         1
64873         1
27636         1
Name: count, dtype: int64
----------------------------------------
Value counts for Destination_Port for label 'PortScan':
Destination_Port
80      373
21      244
22      243
443     240
444     209
       ... 
1051    155
2000     38
5901     30
0         6
123       1
Name: count, Length: 1000, dtype: int64
----------------------------------------
Value counts for Destination_Port for label 'FTP-Bruteforce':
Destination_Port
21    7934
80       1
Name: count, dtype: int64
----------------------------------------
Value counts for 

In [19]:
# map destination port and destination port to 1-6 numbers
def map_port(port):
    if port == 21:
        return 1  # FTP
    elif port == 22:
        return 2  # SSH
    elif port == 53:
        return 3  # DNS
    elif port == 80:
        return 4  # HTTP
    elif port == 443:
        return 5  # HTTPS
    else:
        return 6  # Other

df['Source_Port'] = df['Source_Port'].apply(map_port)
df['Destination_Port'] = df['Destination_Port'].apply(map_port)

# Reorder columns by moving the label column to the last position
cols = list(df.columns)
cols.append(cols.pop(cols.index('Label')))
df = df[cols]

print(df['Destination_Port'].value_counts())
print(df['Source_Port'].value_counts())

Destination_Port
4    448275
6    299424
3    269821
5    144715
1      9541
2      9110
Name: count, dtype: int64
Source_Port
6    1058984
5      81882
4      37282
2       1353
1       1265
3        120
Name: count, dtype: int64


In [21]:
print(df['Label'].value_counts())

Label
Benign            626333
DoS               251712
PortScan          158804
DDoS              128025
FTP-Bruteforce      7935
SSH-Bruteforce      5897
Web-Bruteforce      1507
XSS                  652
SQLi                  21
Name: count, dtype: int64


In [20]:
df

,Source_Port,Destination_Port,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,...,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label
0,4,6,7182757,1,5,6.0,30.0,6.0,6.0,6.00,...,20,45201.000,0.000,45201.0,45201.0,7137556.0,0.00,7137556.0,7137556.0,Benign
1,4,6,938509,1,5,6.0,30.0,6.0,6.0,6.00,...,20,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,Benign
2,6,6,119995732,123,0,0.0,0.0,0.0,0.0,0.00,...,0,3753432.167,5183854.063,13200000.0,1.0,13100000.0,6507819.26,26300000.0,6484077.0,Benign
3,6,5,142954,10,6,553.0,3949.0,196.0,0.0,55.30,...,20,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,Benign
4,4,6,2548725,1,6,6.0,36.0,6.0,6.0,6.00,...,20,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182470,6,4,5005388,5,3,599.0,2021.0,599.0,0.0,119.80,...,32,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,SQLi
1182471,6,4,5016759,5,5,599.0,4149.0,599.0,0.0,119.80,...,32,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,SQLi
1182472,6,4,5009163,4,4,599.0,2021.0,599.0,0.0,149.75,...,32,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,SQLi
1182473,6,4,73,1,1,0.0,0.0,0.0,0.0,0.00,...,32,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,SQLi


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1180886 entries, 0 to 1182474
Data columns (total 79 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   Source_Port                  1180886 non-null  int64  
 1   Destination_Port             1180886 non-null  int64  
 2   Flow_Duration                1180886 non-null  int64  
 3   Total_Fwd_Packets            1180886 non-null  int64  
 4   Total_Backward_Packets       1180886 non-null  int64  
 5   Total_Length_of_Fwd_Packets  1180886 non-null  float64
 6   Total_Length_of_Bwd_Packets  1180886 non-null  float64
 7   Fwd_Packet_Length_Max        1180886 non-null  float64
 8   Fwd_Packet_Length_Min        1180886 non-null  float64
 9   Fwd_Packet_Length_Mean       1180886 non-null  float64
 10  Fwd_Packet_Length_Std        1180886 non-null  float64
 11  Bwd_Packet_Length_Max        1180886 non-null  float64
 12  Bwd_Packet_Length_Min        1180886 non-null  

In [28]:
df['Label'].value_counts()

,count
Label,
Benign,626333
DoS,251712
PortScan,158804
DDoS,128025
FTP-Bruteforce,7935
SSH-Bruteforce,5897
Web-Bruteforce,1507
XSS,652
SQLi,21


In [26]:
# save dataframe to csv file
import os

output_dir = '/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/Dataset'
output_path = os.path.join(output_dir, 'TrainRfWeb.csv')

os.makedirs(output_dir, exist_ok=True)

df.to_csv(output_path, index=False)

print(f"DataFrame saved to {output_path}")

DataFrame saved to /content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/Dataset/TrainRfWeb.csv
